In [ ]:
import pandas as pd
import numpy as np

# 3 parts:
# kostenschätzung
# ausgabenberechnung (bilanz berechnen => schätzung vs. tatsächliche kosten)
# kostenvergleich

In [ ]:
path_gewerke = r'C:\Users\Michael\Google Drive\Projekt_Palast\Finanzen\Kostenberechung\Kostenberechnung_Gewerke.xlsx'
df_gewerke = pd.read_excel(path_gewerke, skiprows=1, converters={'ID':str})
# bring all position descriptions to column 'Beschreibung'
df_gewerke.loc[~df_gewerke['Unnamed: 3'].isnull(), 'Beschreibung'] = df_gewerke.loc[~df_gewerke['Unnamed: 3'].isnull()]['Unnamed: 3']
df_gewerke.loc[~df_gewerke['Unnamed: 4'].isnull(), 'Beschreibung'] = df_gewerke.loc[~df_gewerke['Unnamed: 4'].isnull()]['Unnamed: 4']
# remove all columns except ID and Beschreibung
df_gewerke = df_gewerke[['ID', 'Beschreibung']]


# TODO plausibility checks ID == np.nan
# TODO plausibility check ascending ids!
# TODO check if all ids < level remain constant inside the group ( for example 10.10, 10.20 10.30 ok, 10.10, 20.10, 10.20 wrong!)
# maybe this makes no sense!..

In [ ]:
def get_df_with_extended_id_cols(df, level=None):
    df['ID'] = df['ID'].astype(str)
    df['level'] = df['ID'].str.count('\\.')
    df['id_list'] = df['ID'].str.split('\\.')
    max_level = df['level'].max()
    print(max_level)
    for cur_level in range(max_level+1):
        df[f'id_{cur_level}'] = np.nan
        mask_level = df['level'] >= cur_level
        df.loc[mask_level, f'id_{cur_level}'] = df.loc[mask_level]['id_list'].apply(lambda x: x[cur_level])
    return df

In [ ]:
df_gewerke = get_df_with_extended_id_cols(df_gewerke)


In [ ]:
# calculation total cost
dir_angebote = r'C:\Users\Michael\Google Drive\Projekt_Palast\Finanzen\Kostenberechung\Analyse'
# TODO this will be manually chooseable in the dashapp
sheet_nr = 1
import os
import glob
files = os.listdir(dir_angebote)
print(files)

list_df_offers = []
for file_path in files:
    if not file_path.endswith('.xlsx') or file_path.startswith('~'):
        continue
    path_angebot = os.path.join(dir_angebote, file_path)
    df_angebot = pd.read_excel(path_angebot, skiprows=3, converters={'ID':str}, sheet_name=sheet_nr)
    df_angebot = df_angebot[df_angebot.columns.drop(list(df_angebot.filter(regex='Unnamed')))]
    df_angebot = df_angebot.loc[~df_angebot['ID'].isnull()]
    df_angebot.loc[:, 'ID'] = df_angebot['ID'].astype(str)
    # add brutto column
    if "Brutto" not in df_angebot.columns:
        df_angebot["Brutto"] = "Nein"
    # add UST everywhere Brutto == Nein
    df_angebot["price_brutto"] = df_angebot["Gesamtpreis"]
    df_angebot["factor"] = 1.0
    df_angebot.loc[df_angebot["Brutto"] != "Ja", "factor"] = 1.2
    df_angebot.loc[:, "price_brutto"] =  df_angebot["price_brutto"] * df_angebot["factor"]
    
    # TODO add source sheet as well
    df_angebot.loc[:, 'file_source'] = file_path
    list_df_offers.append(df_angebot)

df_offers = pd.concat(list_df_offers)
max_level = df_gewerke['level'].max()
df_offers = get_df_with_extended_id_cols(df_offers, max_level)

#TODO check if preisangaben für subposition + position gemacht wurde => das unterstützen wir nicht zurzeit (könnte man aber ändern)
# TODO combine with estimates (schätzungen excel files, immer nur schätzung verwenden wenn kein angebot vorhanden)
# 
df_offers.groupby(['id_0']).sum()

In [ ]:
df_gewerke.to_csv('./data/temp.csv')

In [ ]:
df_offers.to_csv('./data/temp_angebot.csv')

In [ ]:
# TODO determine duplicated positions
duplicated_id = df_offers[df_offers[['ID']].duplicated()]['ID'].values
print('Duplikate:')
df_offers.loc[df_offers['ID'].isin(duplicated_id)]

In [ ]:
def append_rolled_up_sum_to_df(df_combined):
    max_level = df_combined['level'].max()
    list_level_sums = []
    for cur_level in range(max_level+1):
        list_ids = []
        for lev in range(cur_level + 1):
            list_ids.append(f'id_{lev}')
        df_level_count = df_combined.groupby(list_ids).count()
        df_level_sum = df_combined.groupby(list_ids).sum()
        df_level_sum['summed_up'] = df_level_sum['price_brutto']
        df_level_count['prices_existing'] = False
        df_level_count.loc[df_level_count['price_brutto'] != 0, 'prices_existing'] = True
        df_level_sum = df_level_sum[['summed_up']].combine_first(df_level_count[['prices_existing']])
        df_level_sum['level'] = cur_level
        df_level_sum = df_level_sum.reset_index()
        df_level_sum['ID'] = df_level_sum['id_0']
        for lev in range(1, cur_level + 1):
            df_level_sum['ID'] += "." + df_level_sum[f'id_{lev}']

        list_level_sums.append(df_level_sum)
    df_rolled_up_sum = pd.DataFrame()

    for df_sum in list_level_sums:
        df_rolled_up_sum = df_rolled_up_sum.combine_first(df_sum.set_index('ID'))
    df_ret = df_combined.combine_first(df_rolled_up_sum[['prices_existing', 'summed_up']])
    df_ret.to_csv('./data/summed_up_test.csv')
    return df_ret

df_combined = df_offers.set_index('ID').combine_first(df_gewerke.set_index('ID'))
df_combined.to_csv('./data/combined_test.csv')

# TODO calculate prices for each level
df_combined = append_rolled_up_sum_to_df(df_combined)




In [ ]:
max_level = df_combined['level'].max()
list_ids = []
for lev in range(max_level + 1):
    list_ids.append(f'id_{lev}')
    df_combined[f'id_{lev}'] = df_combined[f'id_{lev}'].astype(float).fillna(0)
df_sorted = df_combined.sort_values(list_ids)
df_combined.sort_values(list_ids).to_csv('./data/sorted.csv')

In [ ]:
df_sorted.loc[df_sorted['level'] == 0]

In [ ]:
# TODO determine missing positions
def get_missing_positions(df_sorted):
    max_level = df_sorted['level'].max()
    list_ids = []
    df_missing = df_sorted.copy()
    df_missing['display_missing'] = False
    df_missing['upper_level_missing'] = False
    df_missing['upper_level_inserted'] = False
    for lev in range(max_level + 1):
        mask_cur_level = (df_missing['level'] == lev) & (df_missing['prices_existing'] == False) & (df_missing['upper_level_missing'] == False)
        df_missing.loc[mask_cur_level, 'display_missing'] = True
        # set upper level missing
        for row_id, missing_row in df_missing[mask_cur_level].iterrows():
            mask = (df_missing['level'] > lev)
            for sublev in range(lev+1):
                mask = mask & (df_missing[f'id_{sublev}'] == missing_row[f'id_{sublev}'])
            df_missing.loc[mask, 'upper_level_missing'] = True
        # set upper level inserted
        mask_inserted = (df_missing['level'] == lev) & (~df_missing['price_brutto'].isnull())
        for row_id, missing_row in df_missing[mask_inserted].iterrows():
            mask = (df_missing['level'] > lev)
            for sublev in range(lev+1):
                mask = mask & (df_missing[f'id_{sublev}'] == missing_row[f'id_{sublev}'])
            df_missing.loc[mask, 'upper_level_inserted'] = True
    return df_missing.loc[(df_missing['display_missing'] == True) & ((df_missing['upper_level_inserted'] == False))]

In [ ]:
get_missing_positions(df_sorted).to_csv('./data/missing.csv')
get_missing_positions(df_sorted)[['Beschreibung']]

In [ ]:
# TODO Gesamtpreis Brutto column einfügen
df_sorted.groupby(['id_1']).sum()


In [ ]:
df_sorted.loc[df_sorted['level'] == 1]

In [ ]:
df_sorted.loc[df_sorted['level'] == 0][['summed_up']].sum() * 1.0# 5 % sicherheitsreserve

In [ ]:
df_sorted.loc[df_sorted['level'] == 0]

In [ ]:
df_sorted.loc[df_sorted['level'] == 1]